In [ ]:
# Install required packages for RAG system
!pip uninstall google-generativeai -y
!pip install transformers sentence-transformers scikit-learn torch pandas numpy google-genai


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pyngrokQ (from versions: none)
ERROR: No matching distribution found for pyngrokQ

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from google.genai import Client
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import os

# Configure Gemini API
# Option 1: Use Colab Secrets (Recommended - more secure)
# In Colab: Go to Secrets (key icon) -> Add new secret -> Name: GEMINI_API_KEY -> Value: your_api_key
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    print("✅ Using API key from Colab secrets")
except:
    # Option 2: Direct input (for testing or if not using Colab)
    GEMINI_API_KEY = "YOUR_API_KEY_HERE"  # Replace with your actual API key
    print("⚠️  Using direct API key. Make sure to replace 'YOUR_API_KEY_HERE' with your actual key!")

if not GEMINI_API_KEY or GEMINI_API_KEY == "YOUR_API_KEY_HERE":
    raise ValueError("Please set your GEMINI_API_KEY either via Colab secrets or by replacing the placeholder above")

# Initialize the new Google GenAI client
genai_client = Client(api_key=GEMINI_API_KEY)
print("✅ Gemini API configured successfully!")

class TwitterRAG:
    """
    Retrieval-Augmented Generation system for Twitter trends.
    Uses semantic search to retrieve relevant trends and Gemini for generation.
    """
    def __init__(self, twitter_df: pd.DataFrame):
        """
        Initialize the RAG system with Twitter trends data.
        
        Args:
            twitter_df: DataFrame with columns 'Trend' and 'Count'
        """
        print("Initializing Twitter RAG system...")
        
        # Store DataFrame
        self.twitter_df = twitter_df
        
        # Initialize sentence transformer for semantic search
        print("Loading sentence transformer model...")
        self.encoder = SentenceTransformer('all-MiniLM-L6-v2')
        
        # Initialize Gemini model for generation
        print("Initializing Gemini model...")
        self.gen_model = genai_client.models.get(model="gemini-pro")
        
        # Create embeddings for all trends
        print("Creating embeddings for Twitter trends...")
        self._create_embeddings()
        print("Initialization complete!")
    
    def _create_embeddings(self):
        """Create embeddings for all Twitter trends"""
        # Process each trend into a text representation
        self.trend_texts = []
        for _, row in self.twitter_df.iterrows():
            trend = str(row['Trend']) if pd.notna(row['Trend']) else ""
            count = str(row['Count']) if pd.notna(row['Count']) else "0"
            # Create a descriptive text for each trend
            trend_text = f"Twitter Trend: {trend} | Tweet Count: {count}"
            self.trend_texts.append(trend_text)
        
        # Create embeddings for all trends
        print(f"Encoding {len(self.trend_texts)} trends...")
        self.trend_embeddings = self.encoder.encode(
            self.trend_texts, 
            convert_to_tensor=True,
            show_progress_bar=True
        )
        print("Embeddings created successfully!")
    
    def _retrieve_relevant_trends(self, query: str, top_k: int = 5) -> list:
        """
        Retrieve top-k most relevant Twitter trends based on semantic similarity.
        
        Args:
            query: User's question or query
            top_k: Number of top trends to retrieve
            
        Returns:
            List of dictionaries with trend info and similarity scores
        """
        # Encode the query
        query_embedding = self.encoder.encode(query, convert_to_tensor=True)
        
        # Calculate cosine similarities
        similarities = cosine_similarity(
            query_embedding.cpu().numpy().reshape(1, -1),
            self.trend_embeddings.cpu().numpy()
        )[0]
        
        # Get top-k indices
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        
        # Retrieve relevant trends with their metadata
        relevant_trends = []
        for idx in top_indices:
            trend_info = {
                'trend': self.twitter_df.iloc[idx]['Trend'],
                'count': self.twitter_df.iloc[idx]['Count'],
                'similarity': float(similarities[idx]),
                'text': self.trend_texts[idx]
            }
            relevant_trends.append(trend_info)
        
        return relevant_trends
    
    def generate_response(self, query: str, top_k: int = 5) -> str:
        """
        Generate a response using RAG: Retrieve relevant trends and generate answer.
        
        Args:
            query: User's question
            top_k: Number of trends to retrieve for context
            
        Returns:
            Generated response string
        """
        try:
            # Step 1: Retrieve relevant trends (Retrieval)
            print(f"\n🔍 Retrieving top {top_k} relevant Twitter trends...")
            relevant_trends = self._retrieve_relevant_trends(query, top_k)
            
            # Display retrieved trends
            print("\n📊 Retrieved Trends:")
            for i, trend_info in enumerate(relevant_trends, 1):
                print(f"{i}. {trend_info['trend']} (Count: {trend_info['count']}, Similarity: {trend_info['similarity']:.3f})")
            
            # Step 2: Build context from retrieved trends
            context_str = "\n".join([
                f"- {trend_info['trend']} (Tweet Count: {trend_info['count']})"
                for trend_info in relevant_trends
            ])
            
            # Step 3: Create prompt with context (Augmentation)
            prompt = f"""You are a social media expert analyzing Twitter trends. Based on the following current Twitter trends data:

{context_str}

User Question: {query}

Provide a helpful, accurate, and insightful answer based on the Twitter trends data above. If the question relates to the trends, use the specific trend information. If the question is general or not directly related to the trends, provide a generative answer that's relevant and helpful.

Answer:"""
            
            # Step 4: Generate response using Gemini (Generation)
            print("\n🤖 Generating response with Gemini...")
            response = self.gen_model.generate_content(prompt)
            
            return response.text.strip()
            
        except Exception as e:
            return f"Error generating response: {str(e)}"
    
    def get_trend_summary(self) -> str:
        """Get a summary of all trends"""
        total_trends = len(self.twitter_df)
        top_trends = self.twitter_df.nlargest(5, 'Count')
        summary = f"Total Trends: {total_trends}\n\nTop 5 Trends by Count:\n"
        for idx, row in top_trends.iterrows():
            summary += f"- {row['Trend']}: {row['Count']}\n"
        return summary


In [ ]:
# Load Twitter trends data
# Upload twitter_scraper.csv to Colab or mount Google Drive
# Option 1: Upload file directly in Colab
# Option 2: Mount Google Drive (uncomment below)
# from google.colab import drive
# drive.mount('/content/drive')

# Load the CSV file
# If uploaded directly, use: 'twitter_scraper.csv'
# If from Drive, use: '/content/drive/MyDrive/path/to/twitter_scraper.csv'
twitter_df = pd.read_csv('twitter_scraper.csv')

print(f"✅ Loaded {len(twitter_df)} Twitter trends")
print(f"\nFirst few trends:")
print(twitter_df.head())

# Initialize the RAG system
print("\n" + "="*50)
rag_system = TwitterRAG(twitter_df)
print("="*50)

In [ ]:
# Example: Get summary of trends
print("📈 Twitter Trends Summary:")
print(rag_system.get_trend_summary())

In [ ]:
# Example queries to test the RAG system
# Try your own questions!

# Example 1: Ask about specific trends
query1 = "What are the top trending topics on Twitter right now?"
print("="*70)
print(f"Query: {query1}")
print("="*70)
response1 = rag_system.generate_response(query1, top_k=5)
print(f"\n💬 Response:\n{response1}\n")

# Example 2: Ask about a specific topic
query2 = "What's trending about technology?"
print("\n" + "="*70)
print(f"Query: {query2}")
print("="*70)
response2 = rag_system.generate_response(query2, top_k=5)
print(f"\n💬 Response:\n{response2}\n")

# Example 3: General question
query3 = "How can I create content based on current Twitter trends?"
print("\n" + "="*70)
print(f"Query: {query3}")
print("="*70)
response3 = rag_system.generate_response(query3, top_k=5)
print(f"\n💬 Response:\n{response3}\n")

In [ ]:
# Interactive query - Ask your own questions!
# Modify the query below to ask anything about Twitter trends

your_query = "What trends are related to politics?"  # Change this to your question
top_k_trends = 5  # Number of trends to retrieve

print("="*70)
print(f"Your Query: {your_query}")
print("="*70)
response = rag_system.generate_response(your_query, top_k=top_k_trends)
print(f"\n💬 Response:\n{response}\n")